## Hyperparameter Optimisation

In [1]:
import numpy as np

import utils.helpers as h
from utils.data_loader import get_stock_returns

from algorithms.dfo import DFO
from algorithms.pso import PSO
from algorithms.equal_weights import EqualWeights
from algorithms.monte_carlo import MonteCarlo

In [2]:
STARTING_CAPITAL = 10_000
RISK_FREE_RATE = 5.25 / 100
SPLIT_DATE = "2023-11-05"

#N = 50
data = get_stock_returns()
#data = data.iloc[:, :N]


in_sample = data[data.index <= SPLIT_DATE] # Data used to "train"
out_of_sample = data[data.index > SPLIT_DATE] # Data used to "test"

#print(in_sample.head(1))
#print(out_of_sample.head(1))

Loading stock return data...


Sharpe Ratio = (expected return of portfolio – risk-free rate) / portfolio volatility

Maximising this predicts the portfolio with the best risk-adjusted returns.

In [3]:
def _sharpe(weights, returns):
    """Calculates the yearly Sharpe ratio (fitness) for the portfolio.
       Converts daily returns and covariance to trading-yearly values (252 days).

    Args:
        weights (arr): The weight of each asset.
        returns (pd.DataFrame): Daily percentage returns for each asset.

    Returns:
        float: yearly Sharpe ratio 
    """
    po_return = np.sum(returns.mean() * weights) * 252

    covariance = returns.cov() * 252 # Measure of asset trends
    variance = np.dot(weights.T, np.dot(covariance, weights)) # Overall risk
    po_volatility = np.sqrt(variance) # Total risk

    return (po_return - RISK_FREE_RATE) / po_volatility

def _fitness_function(weights):
    return _sharpe(weights, in_sample)


In [4]:
population = 100
ITERATIONS = 1000
CARDINALITY = 10
tickers_amount = data.shape[1]

disturbances = [0.1, 0.05, 0.01, 0.005, 0.001]
final_fitness = []

for d in disturbances:
    current_fitness = []
    for _ in range(3):
        dfo = DFO(
            fitness_function=_fitness_function,
            cardinality=CARDINALITY,
            population=population,
            dimensions=tickers_amount,
            disturbance=d,
            max_iterations=ITERATIONS,
            lower_bound=0,
            upper_bound=1,
        )

        current_best_dfo_fitness, best_dfo_position = dfo.run()
        current_fitness.append(current_best_dfo_fitness)

    average_fitness = np.mean(current_fitness)
    final_fitness.append(average_fitness)
    print(f"Average: {average_fitness}")

best_disturbance = disturbances[np.argmax(final_fitness)]
print(f"Best disturbance: {best_disturbance}")


Iteration: 0	Best fly index: 1	Fitness value: 0.9188927887773557
Iteration: 100	Best fly index: 77	Fitness value: 1.3107927212783248
Iteration: 200	Best fly index: 30	Fitness value: 1.3577104973509089
Iteration: 300	Best fly index: 30	Fitness value: 1.3577104973509089
Iteration: 400	Best fly index: 30	Fitness value: 1.3577104973509089
Iteration: 500	Best fly index: 30	Fitness value: 1.3898907444507427
Iteration: 600	Best fly index: 30	Fitness value: 1.3898907444507427
Iteration: 700	Best fly index: 30	Fitness value: 1.3898907444507427
Iteration: 800	Best fly index: 30	Fitness value: 1.3898907444507427
Iteration: 900	Best fly index: 30	Fitness value: 1.3898907444507427
Iteration: 0	Best fly index: 67	Fitness value: 1.002647062922832
Iteration: 100	Best fly index: 64	Fitness value: 1.290475903622026
Iteration: 200	Best fly index: 62	Fitness value: 1.3129464727975833
Iteration: 300	Best fly index: 62	Fitness value: 1.3129464727975833
Iteration: 400	Best fly index: 63	Fitness value: 1.3371

In [9]:
population = 100
ITERATIONS = 1000
CARDINALITY = 10
tickers_amount = data.shape[1]

# No difference below 0.001
disturbances = [0.005, 0.004, 0.003, 0.002, 0.001]
final_fitness = []

for d in disturbances:
    current_fitness = []
    for _ in range(3):
        dfo = DFO(
            fitness_function=_fitness_function,
            cardinality=CARDINALITY,
            population=population,
            dimensions=tickers_amount,
            disturbance=d,
            max_iterations=ITERATIONS,
            lower_bound=0,
            upper_bound=1,
        )

        current_best_dfo_fitness, best_dfo_position = dfo.run()
        current_fitness.append(current_best_dfo_fitness)

    average_fitness = np.mean(current_fitness)
    final_fitness.append(average_fitness)
    print(f"Average: {average_fitness}")

best_disturbance = disturbances[np.argmax(final_fitness)]
print(f"Best disturbance: {best_disturbance}")


Iteration: 0	Best fly index: 33	Fitness value: 0.9827047937815254
Iteration: 100	Best fly index: 92	Fitness value: 1.448823674907805
Iteration: 200	Best fly index: 5	Fitness value: 1.4531134577608964
Iteration: 300	Best fly index: 11	Fitness value: 1.4548747694720912
Iteration: 400	Best fly index: 12	Fitness value: 1.4548896070210018
Iteration: 500	Best fly index: 12	Fitness value: 1.4548896070210018
Iteration: 600	Best fly index: 8	Fitness value: 1.4570937921978198
Iteration: 700	Best fly index: 8	Fitness value: 1.4570937921978198
Iteration: 800	Best fly index: 8	Fitness value: 1.4571049056522916
Iteration: 900	Best fly index: 8	Fitness value: 1.4571049056522916
Iteration: 0	Best fly index: 95	Fitness value: 0.9358311640695659
Iteration: 100	Best fly index: 92	Fitness value: 1.4614198961421536
Iteration: 200	Best fly index: 29	Fitness value: 1.4616520386493808
Iteration: 300	Best fly index: 40	Fitness value: 1.4626417656783395
Iteration: 400	Best fly index: 1	Fitness value: 1.46339712

In [15]:
options = [
    {"c1": 1.5, "c2": 1.5, "w": 0.7},
    # {"c1": 2.0, "c2": 2.0, "w": 0.4},
    # {"c1": 0.5, "c2": 0.5, "w": 0.9},
    # {"c1": 1.5, "c2": 1.5, "w": 0.6},
    # {"c1": 1.5, "c2": 1.5, "w": 0.8},
    # {"c1": 1.7, "c2": 1.3, "w": 0.7},
    # {"c1": 1.3, "c2": 1.7, "w": 0.7},
    # {"c1": 2.0, "c2": 2.0, "w": 0.7},
    # {"c1": 1.0, "c2": 1.0, "w": 0.7},
    # {"c1": 2.0, "c2": 1.0, "w": 0.7},
    # {"c1": 1.0, "c2": 2.0, "w": 0.7},
    {"c1": 1.4, "c2": 1.4, "w": 0.7},
    {"c1": 1.6, "c2": 1.6, "w": 0.7},
    {"c1": 1.6, "c2": 1.4, "w": 0.7},
    {"c1": 1.4, "c2": 1.6, "w": 0.7},
]

final_fitness = []

for o in options:
    current_fitness = []
    for _ in range(3):
        pso = PSO(
            fitness_function=_fitness_function,
            cardinality=CARDINALITY,
            population=population,
            dimensions=tickers_amount,
            max_iterations=ITERATIONS,
            lower_bound=0,
            upper_bound=1,
            options=o,
        )

        current_best_pso_fitness, best_pso_position = pso.run()
        current_fitness.append(current_best_dfo_fitness)

    average_fitness = np.mean(current_fitness)
    final_fitness.append(average_fitness)
    print(f"Average: {average_fitness}")

best_options = options[np.argmax(final_fitness)]
print(f"Best options: {best_options}")

2025-11-26 19:15:44,664 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.7}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=-1.38
2025-11-26 19:19:10,612 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -1.3787727913091408, best pos: [0.         0.         0.         0.         0.         0.
 0.         0.         0.20447282 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.52872138 0.         0.         0.         0.
 0.30550124 0.         0.         0.         0.         0.74422731
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.44468642
 0.         0.         0.         0.         0.         0.
 0.      

Final portfolio after enforcing constraints again: [0.         0.         0.         0.         0.         0.
 0.         0.         0.03507372 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.09069286 0.         0.         0.         0.
 0.05240337 0.         0.         0.         0.         0.12765911
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.07627814
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.16703243 0.         0.         0.
 0.15193984 0.         0.         0.         0.         0.
 0.16746991 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0. 

pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=-1.35
2025-11-26 19:22:33,811 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -1.3513500666029195, best pos: [0.         0.67182881 0.         0.         0.         0.
 0.         0.         0.80101349 0.         0.         0.95325288
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.60731302 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.95445325
 0.         0.         0.         0.         0.         0.85223595
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.91404

Final portfolio after enforcing constraints again: [0.         0.08510785 0.         0.         0.         0.
 0.         0.         0.10147307 0.         0.         0.12075889
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.07693493 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.12091095
 0.         0.         0.         0.         0.         0.10796198
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.11579158 0.         0.         0.         0.         0.
 0.11906795 0.         0.         0.         0.         0.
 0.         0.         0.         0.    

pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=-1.36
2025-11-26 19:25:57,107 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -1.3596610964483404, best pos: [0.         0.         0.         0.         0.         0.
 0.         0.         0.92778734 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.30498944 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.4668986
 0.         0.         0.         0.         0.         0.35679051
 0.         0.         0.         0.         0.         0.65436177
 0.         0.         0.69247343 0.         0.         0.
 0.751921

...

Final portfolio after enforcing constraints again: [0.         0.         0.         0.         0.         0.
 0.         0.         0.09178028 0.         0.         0.08800509
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.07593959 0.         0.
 0.         0.09082103 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.12008473 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.10394188
 0.         0.         0.         0.         0.06417809 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.13472504 0.         0.         0.         0.         0.
 0.12935063 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0. 

pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=-1.41
2025-11-26 20:07:15,502 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -1.413929417487617, best pos: [0.         0.         0.         0.         0.         0.
 0.         0.         0.60762509 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.65982457 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.63233439
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.8701463
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.80715585 0.         0.         0.
 0.85303523 0.    

Final portfolio after enforcing constraints again: [0.         0.         0.         0.         0.         0.
 0.         0.         0.08202936 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.08907629 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.08536512
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.11746971
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.10896601 0.         0.         0.
 0.11515972 0.         0.         0.         0.         0.
 0.13239771 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0. 

pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=-1.37
2025-11-26 20:10:53,119 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -1.3671235992138846, best pos: [0.         0.         0.         0.         0.         0.
 0.         0.         0.78025576 0.         0.         0.66446345
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.46592892
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.29050483 0.70246739 0.         0.         0.
 0.99547971 0.  

Final portfolio after enforcing constraints again: [0.         0.         0.         0.         0.         0.
 0.         0.         0.12303469 0.         0.         0.10477597
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.07347004
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.04580828 0.11076862 0.         0.         0.
 0.15697229 0.         0.         0.         0.         0.
 0.12196947 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0. 